[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **Fine-Tune YOLO for Threat Detection**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)
[![Scientific Paper](https://img.shields.io/badge/Official-Paper-blue.svg)](<PAPER LINK>)


### **Create Dataset and Annotation**

In [ ]:
# Collect various images of threats like guns from the internet.
# Annotate the images using a tool like Labellerr

### **Convert COCO JSON Annotation to YOLO format**

In [ ]:
!git clone https://github.com/Labellerr/yolo_finetune_utils.git

In [ ]:
from yolo_finetune_utils.coco_yolo_converter.seg_converter import coco_to_yolo_converter

result = coco_to_yolo_converter(
            json_path='./annotation.json',
            images_dir='./dataset',
            output_dir='yolo_format'
            )

### **Train YOLO11 Model on a Custom Dataset**

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

In [ ]:
!yolo task=detect mode=train data="path/to/dataset.yaml" model="yolo11x.pt" epochs=200 imgsz=640 batch=20

### **Inferencing Fine-Tune YOLO model**

In [ ]:
import cv2

# Load the YOLO11 model
model = YOLO('./runs/detect/train3/weights/best.pt', task="detect")

# Open video capture
cap = cv2.VideoCapture("./assests/wep2.mp4")

# Define video writer to save output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference on the frame
    results = model(frame)
    
    # Process each detection
    for result in results[0].boxes:
        # Get box coordinates
        x1, y1, x2, y2 = map(int, result.xyxy[0])
        
        # Get confidence and class
        confidence = float(result.conf[0])
        class_id = int(result.cls[0])
        class_name = " GUN "
        
        if confidence > 0.3:  # Confidence threshold
            # Create translucent overlay
            overlay = frame.copy()
            cv2.rectangle(overlay, (x1, y1), (x2, y2), (0, 0, 255), -1)  # Solid red rectangle
            alpha = 0.4  # Transparency factor
            cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)

            # Add label
            label = f'{class_name} {confidence *100:.2f}%'
            cv2.putText(frame, label, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    # Write the frame with annotation to output video
    output.write(frame)

# Release resources
cap.release()
output.release()
